#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [ ]:
#names here
#Michelle Gelman
#Daphne Lopez

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change those. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [1]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
from math import log2

In [2]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [3]:
def get_data(filename):
    data = []
#    your code goes here
    RM_data = pd.read_csv(filename, sep = ",", header = None)
    f = lambda x: DataPoint(x.iloc[-1], x.iloc[0:19].tolist())
    DataPoints = RM_data.apply(f, axis = 1)
    data = DataPoints.tolist()
    return data

In [4]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [5]:
def make_prediction(tree_root, data_point):
#     your code goes here
    # base case
    if tree_root.is_leaf:
        return int(tree_root.prediction)
    else:
        # compare to threshold and traverse either left or right in tree
        value = data_point.features[int(tree_root.feature_idx)]
        if (value < tree_root.thresh_val):
            return make_prediction(tree_root.left_child, data_point)
        else:
            return make_prediction(tree_root.right_child, data_point) 

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [6]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
#     your code goes here
    for datapoint in data:
        if(datapoint.features[int(feature_idx)] < threshold):
            left_split.append(datapoint)
        else:
            right_split.append(datapoint)
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [7]:
def calc_entropy(data):
    entropy = 0.0
#     your code goes here
    y, n = 0,0
    tot = len(data)
    # base case if data is empty 
    if (tot == 0):
        return 0
    # loop through every datapoint and count y/n
    for datapoint in data:
        if (datapoint.label == 0.0):
            n += 1 
        else:
            y += 1
    # calculate entropy
    if y == 0:
        entropy += - (n/tot)* log2(n/tot)
    elif n == 0:
        entropy += (-(y/tot)* log2(y/tot))
    else:
        entropy += (-(y/tot)* log2(y/tot) - 
                (n/tot)* log2(n/tot))
    return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [8]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = 0.0
#     your code goes here
    parent = calc_entropy(data)
    # sort each datapoint's feature_idx value in ascending order
    sorted_attribute_list = []
    for datapoint in data:
        sorted_attribute_list.append(datapoint.features[feature_idx])
    sorted_attribute_list.sort()
    # loop through the datapoints and find the gain for each thresh avg
    for index in range((len(sorted_attribute_list) - 1)):
        ft_val = sorted_attribute_list[index]
        ft2_val = sorted_attribute_list[index + 1]
        # calculate current threshold based on those values
        thresh = (ft_val + ft2_val) / 2
        left, right = split_dataset(data, feature_idx, thresh)
        en_left = calc_entropy(left)
        en_right = calc_entropy(right)
        gain = parent - (len(left)/len(data))*en_left - (len(right)/len(data))*en_right
        # compare/update best_gain and best_thresh
        if best_info_gain <= gain:
            best_info_gain = gain
            best_thresh = thresh
    return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [9]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = 0
    best_thresh = 0.0
#     your code goes here
    best_gain = 0.0
    cnt = 0
    # loop through every feature and compare its gain
    for feature in data[0].features:
        gain, thresh = calc_best_threshold(data, cnt)
        if  gain > best_gain:
            best_gain = gain
            best_thresh = thresh
            best_feature = cnt
        cnt += 1
    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [10]:
def create_leaf_node(data):
#     your code goes here
    cnt_y = 0
    cnt_n = 0
    # count the class labels for each datapoint
    for datapoint in data:
        if datapoint.label == 1:
            cnt_y += 1
        else:
            cnt_n += 1
    # create a leaf node with the most common label
    leaf_node =  TreeNode()
    leaf_node.is_leaf = True
    # Set prediction; if there is a tie return 1
    if cnt_n > cnt_y:
        leaf_node.prediction = 0
    else:
        leaf_node.prediction = 1
    return leaf_node

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [15]:
def create_decision_tree(data, max_levels):
#     your code goes here
    # base case: if data is empty
    if(data == None):
        return
    # base case: if there is only 1 datapoint
    if (len(data) < 2):
         return create_leaf_node(data)
    # base case: when we've reached the last level, create a leaf
    if(max_levels == 1):
        return create_leaf_node(data)
    # recursive case; find best split, create nodes, and create the child nodes
    else:
        feature_idx, thresh_val = identify_best_split(data)
        print(thresh_val)
        left, right = split_dataset(data, feature_idx, thresh_val)
        node = TreeNode()
        node.is_leaf = False
        node.feature_idx = feature_idx
        node.thresh_val = thresh_val
        node.left_child = create_decision_tree(left, max_levels - 1)
        node.right_child = create_decision_tree(right, max_levels - 1)
        return node

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [16]:
def calc_accuracy(tree_root, data):
#     your code goes here
    corr_cnt = 0
    # loop through each data point and count how many match actual prediction
    for datapoint in data:
        label = datapoint.label
        act_label = make_prediction(tree_root, datapoint)
        if int(label) == int(act_label):
            corr_cnt += 1
    return corr_cnt/len(data)

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [18]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.txt")

#  5-fold cross validation
accuracy_sum = 0.0
fold_size = int(len(d)/5)
avg = 0.0
cnt = 1
start_d = 0
while cnt <= 5:
    # calculate start and end point for this fold
    start_test = start_d * fold_size
    end_d = (cnt * fold_size)
    cnt += 1
    start_d += 1
    
    # partition data into train_set and test_set
    train_set = d[0:int(start_test)].copy() + d[int(end_d):len(d)].copy()
    test_set = d[int(start_test):int(end_d)].copy()

    print ('Training set size:', len(train_set))
    print ('Test set size    :', len(test_set))

    # the timer is just for fun! you will NOT be graded on runtime
    start = time.time()

    # create the decision tree
    tree = create_decision_tree(train_set, 10)

    end = time.time()
    print ('Time taken:', end - start)

    # calculate the accuracy of the tree
    accuracy = calc_accuracy(tree, test_set)
    accuracy_sum += accuracy
    print ('The accuracy on the test set is ', str(accuracy * 100.0))
    #t.printTree()
print ('The average accuracy of this model is ', str(accuracy_sum/5 * 100.0))

tree = create_decision_tree(d[0:20], 10)

make_prediction(tree, d[0])

Training set size: 920
Test set size    : 230


KeyboardInterrupt: 

In [22]:
tree = create_decision_tree(d[0:20], 10)
tree.printTree()
#make_prediction(tree, d[0])

27.0
65.9027985
0.0
0.0
0.0
0.0
0.0
0.0
0.0
44.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.011693
0.0
0.0
0.0
0.0
0.0
0.0
58.5
0.0
0.0
0.0
0.0
0.0
Internal Node:  splits on feature 2 with threshold 27.0
-Internal Node:  splits on feature 8 with threshold 65.9027985
--Internal Node:  splits on feature 0 with threshold 0.0
---Leaf Node:      predicts 1
---Internal Node:  splits on feature 0 with threshold 0.0
----Leaf Node:      predicts 1
----Internal Node:  splits on feature 0 with threshold 0.0
-----Leaf Node:      predicts 1
-----Internal Node:  splits on feature 0 with threshold 0.0
------Leaf Node:      predicts 1
------Internal Node:  splits on feature 0 with threshold 0.0
-------Leaf Node:      predicts 1
-------Internal Node:  splits on feature 0 with threshold 0.0
--------Leaf Node:      predicts 1
--------Internal Node:  splits on feature 0 with threshold 0.0
---------Leaf Node:      predicts 1
---------Leaf Node:      predicts 0
--Leaf Node:      predicts 1
-Internal Node:  splits on fea